In [ ]:
import torch

from utils import *
from dataset_utils import *
from sal_resnet import resnet50
from torch.nn import functional as F

from alexnet import alexnet
from squeezenet import squeezenet1_1
from densenet import densenet121

seed = 1
from sal_resnet import resnet50, resnext50_32x4d, wide_resnet50_2
from madry_models import vit_base_patch16_224 as vit_b_16, deit_base_patch16_224 as deit_b_16

set_seed(seed)
torch.set_default_dtype(torch.float32)
# torchvision.set_image_backend('accimage')
torch.set_float32_matmul_precision('medium')

In [ ]:
img_transform = transforms.Compose([
                    transforms.Resize(224, interpolation=BICUBIC, max_size=None, antialias=None),
                    transforms.CenterCrop(size=(224, 224)),
                    transforms.ToTensor()
                    ])

mask_transform = transforms.Compose([
                    transforms.Resize(224, interpolation=NEAREST, max_size=None, antialias=None),
                    transforms.CenterCrop(size=(224, 224)),
                    ToTensor()
                    ])

In [ ]:
pixel_imagenet = PixelImageNet(IMAGENET_PATH, 
                             PIXEL_IMAGENET_PATH,
                             img_transform=img_transform, mask_transform=mask_transform)

In [ ]:
num_workers = 4*torch.cuda.device_count()
gpu_size = 256*torch.cuda.device_count()
from torch.utils.data import SubsetRandomSampler

pixel_imagenet_loader = torch.utils.data.DataLoader(pixel_imagenet, 
                                                     batch_size=max(gpu_size, 100), 
                                                     num_workers=num_workers, 
                                                     pin_memory=True,
                                                     shuffle=False,
#                                                      sampler=SubsetRandomSampler(indices=),
                                                     drop_last=False)

In [ ]:
def blackout(imgs):
    return torch.zeros_like(imgs, device=imgs.device)

def greyout(imgs):
    return torch.zeros_like(imgs, device=imgs.device) + torch.tensor([[[0.485]], [[0.456]], [[0.406]]], device=imgs.device)

In [ ]:
from efficientnet import efficientnet_b0
from squeezenet import squeezenet1_1
from mobilenet import mobilenet_v2, mobilenet_v3_large
from densenet import densenet121
model_list = [
              (resnet50, 'resnet50'), 
#               (deit_b_16, 'deit_b_16'),
              (wide_resnet50_2, 'wide_resnet50'),
              (alexnet, 'AlexNet'),
              (efficientnet_b0, 'EfficientNet'),
              (mobilenet_v3_large, 'MobileNet'),
              (squeezenet1_1, 'SqueezeNet'),
              (densenet121, 'DenseNet'),
             ]
models = [(MyDataParallel(model_type(pretrained=True)).to(device).eval(), model_name) for model_type, model_name in model_list]
models.append((MyDataParallel(timm.create_model('resnet50', pretrained=True)).to(device).eval(), 'ResNet50_timm'))
baselines = [(blackout, "Blackout"),
            (greyout, "Greyout"),
            ([1000, 1000],"Layer mask")]
normalizer = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


In [ ]:
all_clean_preds = defaultdict(list)
all_masked_preds = dict([(model_name, defaultdict(list)) for _, model_name in models])
all_sqmasked_preds = dict([(model_name, defaultdict(list)) for _, model_name in models])

all_labels = []
hi, wi = torch.arange(224), torch.arange(224)
for i, (imgs, masks, labels) in enumerate(pixel_imagenet_loader):
    all_labels += list(labels.numpy())
    
    sq_masks = torch.cat((masks[:,:,112:], masks[:,:,:112]), dim=2)
    sq_masks = torch.cat((sq_masks[:,:,:,112:], sq_masks[:,:,:,:112]), dim=3)
    
    for model, model_name in models:
        with torch.no_grad():
            clean_logits = model(normalizer(imgs))
            clean_preds = list(clean_logits.argmax(-1).cpu().numpy())
            all_clean_preds[model_name] += clean_preds
            for b, bname in baselines:
                if isinstance(b, list):
                    logits = model((normalizer(imgs)*(1-masks), 1-masks, b))
                    sq_logits = model((normalizer(imgs)*(1-sq_masks), 1-sq_masks, b))
                else:
                    logits = model(normalizer(imgs*(1-masks) + b(imgs)*masks))
                    sq_logits = model(normalizer(imgs*(1-sq_masks) + b(imgs)*sq_masks))
                all_masked_preds[model_name][bname] += list(logits.argmax(-1).cpu().numpy())
                all_sqmasked_preds[model_name][bname] += list(sq_logits.argmax(-1).cpu().numpy())
        if i % 5 == 0 and i != 0:
            hits = (np.array(all_labels) == np.array(all_clean_preds[model_name]))
            blackout_hits = (np.array(all_masked_preds[model_name]['Blackout']) == np.array(all_labels))
            greyout_hits = (np.array(all_masked_preds[model_name]['Greyout']) == np.array(all_labels))
            lm_hits = (np.array(all_masked_preds[model_name]['Layer mask']) == np.array(all_labels))
            sq_blackout_hits = (np.array(all_sqmasked_preds[model_name]['Blackout']) == np.array(all_labels))
            sq_greyout_hits = (np.array(all_sqmasked_preds[model_name]['Greyout']) == np.array(all_labels))
            sq_lm_hits = (np.array(all_sqmasked_preds[model_name]['Layer mask']) == np.array(all_labels))
            print(i, model_name, blackout_hits[hits].mean(), greyout_hits[hits].mean(), lm_hits[hits].mean(),
                  sq_blackout_hits[hits].mean(), sq_greyout_hits[hits].mean(), sq_lm_hits[hits].mean())
        
desc = 'more_models'
with open(f'./results/pixel_imgnet_clean_preds_{desc}.pkl','wb+') as fp:
    pickle.dump(all_clean_preds, fp)
with open(f'./results/pixel_imgnet_masked_preds_{desc}.pkl','wb+') as fp:
    pickle.dump(all_masked_preds, fp)
with open(f'./results/pixel_imgnet_labels_{desc}.pkl','wb+') as fp:
    pickle.dump(all_labels, fp)
with open(f'./results/pixel_imgnet_broken_masked_preds_{desc}.pkl','wb+') as fp:
    pickle.dump(all_sqmasked_preds, fp)

In [1]:
len([0.07628247141838074, 0.0010295228566974401, 0.9682424664497375, 0.0011779558844864368, 0.0006238011410459876, 0.0002019949461100623, 0.016992932185530663, 0.00033032556530088186, 0.00017111783381551504, 0.9996830224990845, 0.0006133546703495085, 0.20553283393383026, 0.004495131317526102, 0.9992827773094177, 0.00017904418928083032, 0.0009415739332325757, 0.01064334437251091, 0.0012078683357685804, 0.04698536545038223, 0.0016126359114423394, 0.05432221665978432, 0.00041419744957238436, 0.1888342648744583, 0.00017133040819317102, 0.9997829794883728, 0.578202486038208, 0.0002259588218294084, 0.0005777108599431813, 0.00017018870858009905, 0.006988284643739462, 0.00016907411918509752, 0.00037169994902797043, 0.00019943817460443825, 0.00028538319747895, 0.043671123683452606, 0.329556941986084, 0.001676948624663055, 0.4916922152042389, 0.00019213533960282803, 0.00334983435459435, 0.00018890874343924224, 0.014709437265992165, 0.9973840117454529, 0.002113172085955739, 0.9869235157966614, 0.32789960503578186, 0.00017053876945283264, 0.000171805324498564, 0.00018436377285979688, 0.00016964820679277182, 0.00016776402480900288, 0.4895072877407074, 0.041694264858961105, 0.2036731094121933, 0.0002453561464790255, 0.0002560508728493005, 0.06516001373529434, 0.00021448977349791676, 0.00017531092453282326, 0.000832237652502954, 0.9092817902565002, 0.026367006823420525, 0.13842642307281494, 0.12105128914117813, 0.034776534885168076, 0.9997093081474304, 0.0001867454993771389, 0.00018578395247459412, 0.0008968145702965558, 0.734694242477417, 0.00021483896125573665, 0.960934042930603, 0.0012310674646869302, 0.00024035279056988657, 0.0002657945442479104, 0.17620934545993805, 0.048088837414979935, 0.0004814622225239873, 0.0001877524919109419, 0.0002697138406801969, 0.006494992878288031, 0.025316163897514343, 0.00403169309720397, 0.5613287687301636, 0.01422441191971302, 0.0003400530549697578, 0.0001698557025520131, 0.3684725761413574, 0.0001715542166493833, 0.3545079529285431, 0.0012368266470730305, 0.00017118392861448228, 0.00020038893853779882, 0.00017136993119493127, 0.002288657706230879, 0.0001728327333694324, 0.030025579035282135, 0.0002344874810660258, 0.05957170948386192, 0.00975550152361393, 0.9884695410728455, 0.004594963043928146, 0.0038921856321394444, 0.0003986684896517545, 0.00017447560094296932, 0.00016876068548299372, 0.004035111982375383, 0.009539932943880558, 0.027976838871836662, 0.0002889583120122552, 0.00019355799304321408, 0.0001825428189476952, 0.00018033123342320323, 0.06457347422838211, 0.00020732745178975165, 0.0004640641564037651, 0.0003330681938678026, 0.9902015328407288, 0.021878834813833237, 0.00017606662004254758, 0.4689672887325287, 0.00024340639356523752, 0.1243075355887413, 0.00020008957653772086, 0.00017757319437805563, 0.0014415517216548324, 0.00019756046822294593, 0.00020668499928433448, 0.0020298324525356293, 0.00031451351242139935, 0.012303678318858147, 0.00021635934535879642, 0.50823974609375, 0.9448611736297607, 0.00019767539924941957, 0.00022459747560787946, 0.00036704589729197323, 0.10291086882352829, 0.01734582707285881, 0.008457662537693977, 0.002468451391905546, 0.9997947812080383, 0.17544712126255035, 0.00017931550974026322, 0.00016921669885050505, 0.9304569363594055, 0.999453604221344, 0.00026238360442221165, 0.000694529851898551, 0.9544090032577515, 0.9774715304374695, 0.03168522194027901, 0.00017041328828781843, 0.057235267013311386, 0.00017394870519638062, 0.00028461372130550444, 0.0044010719284415245, 0.881165087223053, 0.03554654121398926, 0.0003819420235231519, 0.00017935106006916612, 0.1510879099369049, 0.0006547034718096256, 0.002221703762188554, 0.003639203030616045, 0.0005318606272339821, 0.02659866213798523, 0.6933459043502808, 0.1208706796169281, 0.001065717893652618, 0.002186540514230728, 0.0007959271315485239, 0.18188180029392242, 0.0013048010878264904, 0.0010286930482834578, 0.9833005666732788, 0.00035651319194585085, 0.9263242483139038, 0.0025321566499769688, 0.0010744802420958877, 0.006785881705582142, 0.007267664186656475, 0.00018737495702225715, 0.007098740432411432, 0.10204281657934189, 0.00024138903245329857, 0.0002040402323473245, 0.064862921833992, 0.9835739135742188, 0.00026057695504277945, 0.0026151675265282393, 0.0011233766563236713, 0.00016815366689115763, 0.008983471430838108, 0.0007082934025675058, 0.02417817898094654, 0.0002415577764622867, 0.00016999701620079577, 0.00026741650071926415, 0.9988991022109985])

200

In [2]:
len([0.010479722172021866, 0.00016984324611257762, 0.005860438104718924, 0.0001685910247033462, 0.005010728724300861, 0.00017907869187183678, 0.9776766300201416, 0.5184957385063171, 0.00016808284271974117, 0.9971250891685486, 0.00019305810565128922, 0.45532119274139404, 0.0001696205436019227, 0.9996455907821655, 0.00017192561062984169, 0.0019282462308183312, 0.08339335769414902, 0.11751985549926758, 0.010172677226364613, 0.0005620233132503927, 0.02056405134499073, 0.0024615521542727947, 0.9968385696411133, 0.00016829179367050529, 0.6293272972106934, 0.9898290038108826, 0.00023726647486910224, 0.01700589433312416, 0.0003182734362781048, 0.00035492138704285026, 0.0001696637482382357, 0.0002243357157567516, 0.0003087451623287052, 0.0002972409420181066, 0.9880579710006714, 0.002053716452792287, 0.0002663993800524622, 0.008977729827165604, 0.00017888224101625383, 0.00019905745284631848, 0.0001833160495152697, 0.00047867081593722105, 0.9996869564056396, 0.008840608410537243, 0.15872225165367126, 0.08600009977817535, 0.0002247034863103181, 0.00018546661885920912, 0.012696871533989906, 0.00019842415349557996, 0.0002072837669402361, 0.33502915501594543, 0.9976154565811157, 0.4429764151573181, 0.0010702615836635232, 0.00018203322542831302, 0.0004141248355153948, 0.00017094727081712335, 0.0001742613094393164, 0.0009912612149491906, 0.203610360622406, 0.0012997975572943687, 0.5436581373214722, 0.007844572886824608, 0.12552054226398468, 0.9987832903862, 0.00020444394613150507, 0.0002528972690925002, 0.0025537130422890186, 0.02922629565000534, 0.00019016384612768888, 0.004213082604110241, 0.8596726059913635, 0.0003859085263684392, 0.001086844247765839, 0.9972012042999268, 0.000622862484306097, 0.4844830632209778, 0.0001708573108771816, 0.002032412216067314, 0.012487736530601978, 0.00025713362265378237, 0.029175709933042526, 0.030628889799118042, 0.6021994352340698, 0.00017325808585155755, 0.00018165142682846636, 0.045503053814172745, 0.00017624834435991943, 0.08215134590864182, 0.0018949367804452777, 0.00020924686396028847, 0.0002116707037203014, 0.00016854119894560426, 0.561737596988678, 0.0001738788851071149, 0.4401470124721527, 0.00017429288709536195, 0.002380853984504938, 0.055639106780290604, 0.9997091889381409, 0.5056441426277161, 0.000971415953245014, 0.8090801239013672, 0.00020981125999242067, 0.00039363751420751214, 0.0003103955532424152, 0.02910323068499565, 0.0018430690979585052, 0.0009620878845453262, 0.00017601624131202698, 0.00047712266677990556, 0.001924968441016972, 0.02117115631699562, 0.0001704062888165936, 0.0018226789543405175, 0.0002663315972313285, 0.9992746710777283, 0.9718287587165833, 0.00490869116038084, 0.002500362228602171, 0.05534559488296509, 0.015845978632569313, 0.3809053897857666, 0.0001823206985136494, 0.000965100945904851, 0.00018654298037290573, 0.00017573553486727178, 0.01331982109695673, 0.021443303674459457, 0.11949292570352554, 0.00017097075760830194, 0.7901919484138489, 0.21279536187648773, 0.00019196666835341603, 0.02829298935830593, 0.00016902931383810937, 0.0010058681946247816, 0.004616938065737486, 0.0007688503246754408, 0.0004331700038164854, 0.999809205532074, 0.0005406618001870811, 0.004329063463956118, 0.00016969817806966603, 0.9948009252548218, 0.6915777921676636, 0.0003246472042519599, 0.0027304321993142366, 0.004877803381532431, 0.060316987335681915, 0.005256557371467352, 0.00018767373694572598, 0.11942978203296661, 0.000834930338896811, 0.00017972115892916918, 0.5197944045066833, 0.28056037425994873, 0.08489887416362762, 0.00020050937018822879, 0.000178566639078781, 0.00020968726312275976, 0.0006129628163762391, 0.00019988649000879377, 0.08268920332193375, 0.0017208075150847435, 0.014751924201846123, 0.954237163066864, 0.9990330934524536, 0.00020914673223160207, 0.00017710581596475095, 0.00024490788928233087, 0.002360448706895113, 0.10087747126817703, 0.00052444584434852, 0.9673595428466797, 0.0003561585326679051, 0.9996347427368164, 0.0721365287899971, 0.00017738435417413712, 0.012167529202997684, 0.15344613790512085, 0.0002550001081544906, 0.0015994302229955792, 0.0016917901812121272, 0.0001831373228924349, 0.00019475464068818837, 0.0007782502798363566, 0.7722666263580322, 0.00021668904810212553, 0.0004106661072000861, 0.0001907867845147848, 0.0001754534023348242, 0.029261386021971703, 0.0002517586399335414, 0.18677634000778198, 0.0004091424634680152, 0.00017900524835567921, 0.12404481321573257, 0.6933589577674866])

200

In [3]:
len( [0.07628247141838074, 0.0010295228566974401, 0.9682424664497375, 0.0011779558844864368, 0.0006238011410459876, 0.0002019949461100623, 0.016992932185530663, 0.00033032556530088186, 0.00017111783381551504, 0.9996830224990845, 0.0006133546703495085, 0.20553283393383026, 0.004495131317526102, 0.9992827773094177, 0.00017904418928083032, 0.0009415739332325757, 0.01064334437251091, 0.0012078683357685804, 0.04698536545038223, 0.0016126359114423394, 0.05432221665978432, 0.00041419744957238436, 0.1888342648744583, 0.00017133040819317102, 0.9997829794883728, 0.578202486038208, 0.0002259588218294084, 0.0005777108599431813, 0.00017018870858009905, 0.006988284643739462, 0.00016907411918509752, 0.00037169994902797043, 0.00019943817460443825, 0.00028538319747895, 0.043671123683452606, 0.329556941986084, 0.001676948624663055, 0.4916922152042389, 0.00019213533960282803, 0.00334983435459435, 0.00018890874343924224, 0.014709437265992165, 0.9973840117454529, 0.002113172085955739, 0.9869235157966614, 0.32789960503578186, 0.00017053876945283264, 0.000171805324498564, 0.00018436377285979688, 0.00016964820679277182, 0.00016776402480900288, 0.4895072877407074, 0.041694264858961105, 0.2036731094121933, 0.0002453561464790255, 0.0002560508728493005, 0.06516001373529434, 0.00021448977349791676, 0.00017531092453282326, 0.000832237652502954, 0.9092817902565002, 0.026367006823420525, 0.13842642307281494, 0.12105128914117813, 0.034776534885168076, 0.9997093081474304, 0.0001867454993771389, 0.00018578395247459412, 0.0008968145702965558, 0.734694242477417, 0.00021483896125573665, 0.960934042930603, 0.0012310674646869302, 0.00024035279056988657, 0.0002657945442479104, 0.17620934545993805, 0.048088837414979935, 0.0004814622225239873, 0.0001877524919109419, 0.0002697138406801969, 0.006494992878288031, 0.025316163897514343, 0.00403169309720397, 0.5613287687301636, 0.01422441191971302, 0.0003400530549697578, 0.0001698557025520131, 0.3684725761413574, 0.0001715542166493833, 0.3545079529285431, 0.0012368266470730305, 0.00017118392861448228, 0.00020038893853779882, 0.00017136993119493127, 0.002288657706230879, 0.0001728327333694324, 0.030025579035282135, 0.0002344874810660258, 0.05957170948386192, 0.00975550152361393, 0.9884695410728455, 0.004594963043928146, 0.0038921856321394444, 0.0003986684896517545, 0.00017447560094296932, 0.00016876068548299372, 0.004035111982375383, 0.009539932943880558, 0.027976838871836662, 0.0002889583120122552, 0.00019355799304321408, 0.0001825428189476952, 0.00018033123342320323, 0.06457347422838211, 0.00020732745178975165, 0.0004640641564037651, 0.0003330681938678026, 0.9902015328407288, 0.021878834813833237, 0.00017606662004254758, 0.4689672887325287, 0.00024340639356523752, 0.1243075355887413, 0.00020008957653772086, 0.00017757319437805563, 0.0014415517216548324, 0.00019756046822294593, 0.00020668499928433448, 0.0020298324525356293, 0.00031451351242139935, 0.012303678318858147, 0.00021635934535879642, 0.50823974609375, 0.9448611736297607, 0.00019767539924941957, 0.00022459747560787946, 0.00036704589729197323, 0.10291086882352829, 0.01734582707285881, 0.008457662537693977, 0.002468451391905546, 0.9997947812080383, 0.17544712126255035, 0.00017931550974026322, 0.00016921669885050505, 0.9304569363594055, 0.999453604221344, 0.00026238360442221165, 0.000694529851898551, 0.9544090032577515, 0.9774715304374695, 0.03168522194027901, 0.00017041328828781843, 0.057235267013311386, 0.00017394870519638062, 0.00028461372130550444, 0.0044010719284415245, 0.881165087223053, 0.03554654121398926, 0.0003819420235231519, 0.00017935106006916612, 0.1510879099369049, 0.0006547034718096256, 0.002221703762188554, 0.003639203030616045, 0.0005318606272339821, 0.02659866213798523, 0.6933459043502808, 0.1208706796169281, 0.001065717893652618, 0.002186540514230728, 0.0007959271315485239, 0.18188180029392242, 0.0013048010878264904, 0.0010286930482834578, 0.9833005666732788, 0.00035651319194585085, 0.9263242483139038, 0.0025321566499769688, 0.0010744802420958877, 0.006785881705582142, 0.007267664186656475, 0.00018737495702225715, 0.007098740432411432, 0.10204281657934189, 0.00024138903245329857, 0.0002040402323473245, 0.064862921833992, 0.9835739135742188, 0.00026057695504277945, 0.0026151675265282393, 0.0011233766563236713, 0.00016815366689115763, 0.008983471430838108, 0.0007082934025675058, 0.02417817898094654, 0.0002415577764622867, 0.00016999701620079577, 0.00026741650071926415, 0.9988991022109985, 0.013756844215095043, 0.00017032572941388935, 0.11247991025447845, 0.0002066599699901417, 0.04447257146239281, 0.00018552834808360785, 0.28646382689476013, 0.00017037137877196074, 0.00017622800078243017, 0.9586997032165527, 0.00017425647820346057, 0.875663161277771, 0.0002562430454418063, 0.7224245071411133, 0.0004337658465374261, 0.06392581760883331, 0.022209715098142624, 0.10946651548147202, 0.0035754884593188763, 0.0008158195414580405, 0.957511842250824, 0.0009523347252979875, 0.7502733469009399, 0.0001688278280198574, 0.9966557025909424, 0.9798706769943237, 0.0002644692431204021, 0.00028153625316917896, 0.0002118861593771726, 0.0003128618118353188, 0.00016842003969941288, 0.00029821397038176656, 0.0005045657744631171, 0.0004203355056233704, 0.04236947372555733, 0.35766834020614624, 0.0002154590911231935, 0.1075994074344635, 0.00016945807146839797, 0.0020722306799143553, 0.0001793158589862287, 0.0006637387559749186, 0.936017632484436, 0.0002356933691771701, 0.036808449774980545, 0.9929169416427612, 0.00017693029076326638, 0.00039841493708081543, 0.0006965064094401896, 0.00018566583457868546, 0.00017016191850416362, 0.18212126195430756, 0.9990406632423401, 0.302450031042099, 0.08698713034391403, 0.0035995780490338802, 0.08832135051488876, 0.00016905204392969608, 0.00017269303498324007, 0.001283906982280314, 0.07743051648139954, 0.00019085593521595, 0.65196293592453, 0.01011393778026104, 0.02464444190263748, 0.9998185038566589, 0.0002196100977016613, 0.0004635087097994983, 0.004553644917905331, 0.06893040984869003, 0.00025924399960786104, 0.015559532679617405, 0.037802375853061676, 0.0007748640491627157, 0.001326992642134428, 0.9997702240943909, 0.015227770432829857, 0.18208564817905426, 0.00016933985170908272, 0.11510338634252548, 0.004421687219291925, 0.0038796784356236458, 0.01493500079959631, 0.0019271811470389366, 0.011777898296713829, 0.00017032766481861472, 0.00017073971685022116, 0.7800669074058533, 0.0005667720106430352, 0.9241020083427429, 0.0002140595461241901, 0.00019751903892029077, 0.0001690533390501514, 0.00016801968740765005, 0.0002132274239556864, 0.0001843767677200958, 0.029993025586009026, 0.0003234605537727475, 0.13526026904582977, 0.9228995442390442, 0.9998296499252319, 0.11694999039173126, 0.0003323639975860715, 0.001042053336277604, 0.00023393431911244988, 0.00019305774185340852, 0.0011786787072196603, 0.0608174130320549, 0.22429108619689941, 0.00943156611174345, 0.0001703206798993051, 0.00044977973448112607, 0.0001776981953298673, 0.11066580563783646, 0.00028347846819087863, 0.0004188597376924008, 0.001060771755874157, 0.9327500462532043, 0.3215891420841217, 0.00016994141333270818, 0.0017837595660239458, 0.00016724415763746947, 0.010463322512805462, 0.00032835695310495794, 0.00024617533199489117, 0.00021196941088419408, 0.0006160942721180618, 0.0001718834973871708, 0.002601102227345109, 0.01148612517863512, 0.6455836296081543, 0.0005422084359452128, 0.8414853811264038, 0.016512976959347725, 0.00028865845524705946, 0.16784168779850006, 0.006768247112631798, 0.002798116998746991, 0.006091801915317774, 0.9814029932022095, 0.000687706982716918, 0.9998252987861633, 0.0005237818695604801, 0.00017749312974046916, 0.00019894758588634431, 0.9495623111724854, 0.8547112345695496, 0.00023762915225233883, 0.0005922189448028803, 0.008106475695967674, 0.12482154369354248, 0.05623079463839531, 0.00018387642921879888, 0.2708098292350769, 0.00017423421377316117, 0.0001910165010485798, 0.21565626561641693, 0.03474435210227966, 0.010328269563615322, 0.00017526396550238132, 0.00016969058196991682, 0.0008374361786991358, 0.00021206059318501502, 0.05208145081996918, 0.16985739767551422, 0.00016909265832509845, 0.0002711678098421544, 0.12815222144126892, 0.9825383424758911, 0.0007286577601917088, 0.00024530632072128356, 0.0016060806810855865, 0.9053934812545776, 0.0012941040331497788, 0.00017574691446498036, 0.004530385602265596, 0.00031912297708913684, 0.9993520379066467, 0.04162205755710602, 0.00016962007794063538, 0.03213512897491455, 0.012172739952802658, 0.00016801246965769678, 0.04651927202939987, 0.0002534317027311772, 0.0038031733129173517, 0.00020496897923294455, 0.001497384742833674, 0.7613760232925415, 0.00019353583047632128, 0.0001891923020593822, 0.002025442197918892, 0.0001673580554779619, 0.03825912997126579, 0.0026535808574408293, 0.6304445862770081, 0.00020056786888744682, 0.0001976135972654447, 0.017471978440880775, 0.985552966594696])

400